# Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
import pickle

# Import Dataset

In [ ]:
#data review
df_review = pd.read_csv('merged_review_data.csv')
df_tempat = pd.read_csv('merged_restaurant_data.csv')

# Data Preparation

In [ ]:
df_review['resto_id']= df_review['resto_id']*(10**6)
df_review.head()

,user_id,resto_id,review_title,overall_rating,review_date,review_helpful_count,review_text
0,whidiea_wede,1.642423e+15,Tempat ngopi santai,4.0,2021-10-25,1,Tempatnya aku suka karna ada pilihan sofa dan ...
1,laela_marlina,1.642423e+15,Kulineran Malam di Terai,3.8,2021-10-19,1,Coba kulineran malam-malam ke daerah Tubagus I...
2,ExpelliarFood,1.642423e+15,"Good service, good food",4.4,2021-10-03,1,"Datang dalam hari ultahnya RI-2Begitu datang, ..."
3,henie_herliani,1.642423e+15,Terai coffe,4.0,2021-10-01,3,"gyudonnya enak bumbunya terasa, porsinya men..."
4,nyayu_ista,1.642423e+15,Scrumptious lunch in terai🖤,4.2,2021-09-30,3,Siang ini lunch di terai coffeatery yang ada d...


In [ ]:
df_review['resto_id']=df_review['resto_id'].astype(int)
df_review.head()

,user_id,resto_id,review_title,overall_rating,review_date,review_helpful_count,review_text
0,whidiea_wede,1642422817648878,Tempat ngopi santai,4.0,2021-10-25,1,Tempatnya aku suka karna ada pilihan sofa dan ...
1,laela_marlina,1642422817648878,Kulineran Malam di Terai,3.8,2021-10-19,1,Coba kulineran malam-malam ke daerah Tubagus I...
2,ExpelliarFood,1642422817648878,"Good service, good food",4.4,2021-10-03,1,"Datang dalam hari ultahnya RI-2Begitu datang, ..."
3,henie_herliani,1642422817648878,Terai coffe,4.0,2021-10-01,3,"gyudonnya enak bumbunya terasa, porsinya men..."
4,nyayu_ista,1642422817648878,Scrumptious lunch in terai🖤,4.2,2021-09-30,3,Siang ini lunch di terai coffeatery yang ada d...


In [ ]:
df_review['resto_id'] = df_review.resto_id.astype(str).str.pad(width=20, side='right', fillchar='0')

In [ ]:
df_review

,user_id,resto_id,review_title,overall_rating,review_date,review_helpful_count,review_text
0,whidiea_wede,16424228176488780000,Tempat ngopi santai,4.0,2021-10-25,1,Tempatnya aku suka karna ada pilihan sofa dan ...
1,laela_marlina,16424228176488780000,Kulineran Malam di Terai,3.8,2021-10-19,1,Coba kulineran malam-malam ke daerah Tubagus I...
2,ExpelliarFood,16424228176488780000,"Good service, good food",4.4,2021-10-03,1,"Datang dalam hari ultahnya RI-2Begitu datang, ..."
3,henie_herliani,16424228176488780000,Terai coffe,4.0,2021-10-01,3,"gyudonnya enak bumbunya terasa, porsinya men..."
4,nyayu_ista,16424228176488780000,Scrumptious lunch in terai🖤,4.2,2021-09-30,3,Siang ini lunch di terai coffeatery yang ada d...
...,...,...,...,...,...,...,...
8710,sandya_anggraswari,16424023237221570000,Rice Box Enak,4.2,2018-02-24,0,Skeptis dengan rice box sejenis ini karena bi...
8711,rusliani_lia,16424023237221570000,Enak & kenyang!,4.2,2017-12-21,1,Sumpah ini enak bgt!!! \nLokasinya di Jl. Ria...
8712,gracenathh,16424023237221570000,SALTED EGG!!!,3.8,2017-11-06,0,"Akhir2 ini, salted egg lagi populer banget da..."
8713,nesyaadenisaa,16424023237221570000,Sambal Matah Bali yang khas,4.0,2017-10-31,0,Sore tadi nyoba ke salah satu tempat makan ya...


In [ ]:
df_tempat['resto_id']=df_tempat['resto_id'].astype(int)
df_tempat.head()

,resto_id,name,cuisine_type,is_official,lat,long,location,average_price,overall_rating,rating_rasa,...,delivery,smoking_area,outdor_area,24hour,100%_halal,VIP_room,reservation,parking_area,alcohol,vegetarian
0,1642422817648878,Terai Coffeatery,[ Kafe ],True,-6.887844,107.620000,"Jl. Tubagus Ismail Raya No. 175, Dago Atas, B...",Di bawah Rp. 50.000 /orang,4.32,4.3,...,no,yes,yes,no,yes,no,yes,yes,no,no
1,164242282518111,Equator Coffee & Gallery,[ Kafe ],False,-6.899537,107.610458,"Jl. Sulanjana No. 32, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.32,4.2,...,no,yes,yes,no,yes,yes,yes,yes,no,no
2,1642422830535959,One Eighty Coffee and Music,[ Kafe ],True,-6.893863,107.612421,"Jl. Ganeca No. 3, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.11,4.1,...,no,yes,yes,no,yes,yes,yes,yes,no,no
3,1642422843615989,Two Hands Full,[ Kafe ],False,-6.892483,107.612750,"Jl. Ir. H. Juanda No. 113, Dago Bawah, Bandun...",Rp. 50.000 - Rp. 100.000 /orang,4.04,4.3,...,no,yes,yes,no,no,no,no,yes,yes,no
4,164242285404013,Noah's Barn,[ Kafe ],True,-6.887151,107.612748,"Jl. Dayang Sumbi No. 2, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.10,4.0,...,no,yes,yes,no,yes,yes,yes,yes,no,no


In [ ]:
df_tempat['resto_id'] = df_tempat.resto_id.astype(str).str.pad(width=20, side='right', fillchar='0')

In [ ]:
df_tempat

,resto_id,name,cuisine_type,is_official,lat,long,location,average_price,overall_rating,rating_rasa,...,delivery,smoking_area,outdor_area,24hour,100%_halal,VIP_room,reservation,parking_area,alcohol,vegetarian
0,16424228176488780000,Terai Coffeatery,[ Kafe ],True,-6.887844,107.620000,"Jl. Tubagus Ismail Raya No. 175, Dago Atas, B...",Di bawah Rp. 50.000 /orang,4.32,4.3,...,no,yes,yes,no,yes,no,yes,yes,no,no
1,16424228251811100000,Equator Coffee & Gallery,[ Kafe ],False,-6.899537,107.610458,"Jl. Sulanjana No. 32, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.32,4.2,...,no,yes,yes,no,yes,yes,yes,yes,no,no
2,16424228305359590000,One Eighty Coffee and Music,[ Kafe ],True,-6.893863,107.612421,"Jl. Ganeca No. 3, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.11,4.1,...,no,yes,yes,no,yes,yes,yes,yes,no,no
3,16424228436159890000,Two Hands Full,[ Kafe ],False,-6.892483,107.612750,"Jl. Ir. H. Juanda No. 113, Dago Bawah, Bandun...",Rp. 50.000 - Rp. 100.000 /orang,4.04,4.3,...,no,yes,yes,no,no,no,no,yes,yes,no
4,16424228540401300000,Noah's Barn,[ Kafe ],True,-6.887151,107.612748,"Jl. Dayang Sumbi No. 2, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.10,4.0,...,no,yes,yes,no,yes,yes,yes,yes,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5398,16424023237221570000,Mamayu Kitchen,[ Indonesia ],False,-6.914052,107.629578,Sherlock Common Space Jl. LLRE Martadinata No...,Di bawah Rp. 50.000 /orang,3.59,3.5,...,no,yes,no,no,yes,no,no,yes,no,no
5399,16424023296551830000,F.A.T Project,[ Indonesia ],False,-6.906685,107.612264,"Riau Junction, Lantai 3, Foodlife Jl. LLRE Ma...",Di bawah Rp. 50.000 /orang,0.00,0.0,...,no,yes,no,no,yes,no,no,yes,no,no
5400,16424023304674850000,Dapur Nusantara,[ Indonesia ],False,-6.906997,107.620015,"Hashtag Foodstation Jl. Lombok No. 47, Riau, ...",Di bawah Rp. 50.000 /orang,0.00,0.0,...,no,yes,no,no,yes,no,no,yes,no,no
5401,16424023312083990000,Sate Padang Pak Mus,[ Indonesia ],False,-6.906248,107.619068,"Jl. Madura (Samping Taruna Bakti), Riau, Band...",Di bawah Rp. 50.000 /orang,0.00,0.0,...,no,yes,no,no,yes,no,no,no,no,no


In [ ]:
list(df_tempat.keys())  

['resto_id',
 'name',
 'cuisine_type',
 'is_official',
 'lat',
 'long',
 'location',
 'average_price',
 'overall_rating',
 'rating_rasa',
 'rating_suasana',
 'rating_harga_rasa',
 'rating_pelayanan',
 'rating_kebersihan',
 'open_time',
 'payment_method',
 'wifi',
 'delivery',
 'smoking_area',
 'outdor_area',
 '24hour',
 '100%_halal',
 'VIP_room',
 'reservation',
 'parking_area',
 'alcohol',
 'vegetarian']

In [ ]:
df_tempat = df_tempat.drop(['rating_rasa', 'rating_suasana', 'rating_harga_rasa', 'rating_pelayanan', 'rating_kebersihan', 'open_time', 'payment_method', 'wifi', 'delivery',
                'smoking_area', 'outdor_area', '24hour','100%_halal', 'VIP_room','reservation', 'parking_area','alcohol', 'vegetarian'], axis=1)
df_tempat

,resto_id,name,cuisine_type,is_official,lat,long,location,average_price,overall_rating
0,16424228176488780000,Terai Coffeatery,[ Kafe ],True,-6.887844,107.620000,"Jl. Tubagus Ismail Raya No. 175, Dago Atas, B...",Di bawah Rp. 50.000 /orang,4.32
1,16424228251811100000,Equator Coffee & Gallery,[ Kafe ],False,-6.899537,107.610458,"Jl. Sulanjana No. 32, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.32
2,16424228305359590000,One Eighty Coffee and Music,[ Kafe ],True,-6.893863,107.612421,"Jl. Ganeca No. 3, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.11
3,16424228436159890000,Two Hands Full,[ Kafe ],False,-6.892483,107.612750,"Jl. Ir. H. Juanda No. 113, Dago Bawah, Bandun...",Rp. 50.000 - Rp. 100.000 /orang,4.04
4,16424228540401300000,Noah's Barn,[ Kafe ],True,-6.887151,107.612748,"Jl. Dayang Sumbi No. 2, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.10
...,...,...,...,...,...,...,...,...,...
5398,16424023237221570000,Mamayu Kitchen,[ Indonesia ],False,-6.914052,107.629578,Sherlock Common Space Jl. LLRE Martadinata No...,Di bawah Rp. 50.000 /orang,3.59
5399,16424023296551830000,F.A.T Project,[ Indonesia ],False,-6.906685,107.612264,"Riau Junction, Lantai 3, Foodlife Jl. LLRE Ma...",Di bawah Rp. 50.000 /orang,0.00
5400,16424023304674850000,Dapur Nusantara,[ Indonesia ],False,-6.906997,107.620015,"Hashtag Foodstation Jl. Lombok No. 47, Riau, ...",Di bawah Rp. 50.000 /orang,0.00
5401,16424023312083990000,Sate Padang Pak Mus,[ Indonesia ],False,-6.906248,107.619068,"Jl. Madura (Samping Taruna Bakti), Riau, Band...",Di bawah Rp. 50.000 /orang,0.00


In [ ]:
df_tempat.head(5)

,resto_id,name,cuisine_type,is_official,lat,long,location,average_price,overall_rating
0,16424228176488780000,Terai Coffeatery,[ Kafe ],True,-6.887844,107.620000,"Jl. Tubagus Ismail Raya No. 175, Dago Atas, B...",Di bawah Rp. 50.000 /orang,4.32
1,16424228251811100000,Equator Coffee & Gallery,[ Kafe ],False,-6.899537,107.610458,"Jl. Sulanjana No. 32, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.32
2,16424228305359590000,One Eighty Coffee and Music,[ Kafe ],True,-6.893863,107.612421,"Jl. Ganeca No. 3, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.11
3,16424228436159890000,Two Hands Full,[ Kafe ],False,-6.892483,107.612750,"Jl. Ir. H. Juanda No. 113, Dago Bawah, Bandun...",Rp. 50.000 - Rp. 100.000 /orang,4.04
4,16424228540401300000,Noah's Barn,[ Kafe ],True,-6.887151,107.612748,"Jl. Dayang Sumbi No. 2, Dago Bawah, Bandung",Rp. 50.000 - Rp. 100.000 /orang,4.10


In [ ]:
df_review.head()

,user_id,resto_id,review_title,overall_rating,review_date,review_helpful_count,review_text
0,whidiea_wede,16424228176488780000,Tempat ngopi santai,4.0,2021-10-25,1,Tempatnya aku suka karna ada pilihan sofa dan ...
1,laela_marlina,16424228176488780000,Kulineran Malam di Terai,3.8,2021-10-19,1,Coba kulineran malam-malam ke daerah Tubagus I...
2,ExpelliarFood,16424228176488780000,"Good service, good food",4.4,2021-10-03,1,"Datang dalam hari ultahnya RI-2Begitu datang, ..."
3,henie_herliani,16424228176488780000,Terai coffe,4.0,2021-10-01,3,"gyudonnya enak bumbunya terasa, porsinya men..."
4,nyayu_ista,16424228176488780000,Scrumptious lunch in terai🖤,4.2,2021-09-30,3,Siang ini lunch di terai coffeatery yang ada d...


In [ ]:
df_tempat.isnull().sum()

resto_id            0
name                0
cuisine_type      189
is_official         0
lat                 0
long                0
location            0
average_price       0
overall_rating      0
dtype: int64

In [ ]:
df_review.isnull().sum()

user_id                 0
resto_id                0
review_title            0
overall_rating          0
review_date             0
review_helpful_count    0
review_text             0
dtype: int64

In [ ]:
df_tempat.to_csv('df_tempat_new.csv')
df_review.to_csv('df_review_new.csv')

## Case Folding

In [ ]:
import re

# Buat fungsi untuk langkah case folding
def casefolding(mess):
  mess = mess.lower()                               # Mengubah teks menjadi lower case
  mess = re.sub(r'https?://\S+|www\.\S+', '', mess) # Menghapus URL
  mess = re.sub(r'[-+]?[0-9]+', '', mess)           # Menghapus angka
  mess = re.sub(r'[^\w\s]','', mess)                # Menghapus karakter tanda baca
  mess = mess.strip()
  return mess

In [ ]:
raw_sample = df_review['review_text'].iloc[2]
case_folding = casefolding(raw_sample)

print('Raw data\t: ', raw_sample)
print('Case folding\t: ', case_folding)

Raw data	:  Datang dalam hari ultahnya RI-2Begitu datang, disambut oleh kang parkir yang ramah, dan saat masuk ke gedung, gedungnya pun terlihat  cukup luas karena langit-langitnya cukup tinggi, begitu memasuki ruangan say langsung disambut oleh kedua pegawainyaSaya memesan kopi susu terai, fish and chips, chicken quesadilla dan terai japanese pancake di next ordernyaMari kita review satu per satuUntuk kopi susu terai, rasa kopi susunya terasa segar, dan saya minta untuk lebih creamy jugaFish and chipsnya menurut saya cukup oke untuk ukuran cafeIkan dan tepungnya ke mix dengan baik,Nilai plus dari fish and chipsnya adalah saus tar-tarnya, karena sangat pas untuk fish and chipsnyaChicken Quesadillanya disajikan dengan cukup oke, tortillanya renyah dan isiannya menurutku cukup pasNah untuk terai japanese pancakenya saya minta request ke pegawainya untuk dikasih lilin (doang) tadinya, Tidak taunya malah dikasih lebih 😂Pancake ini diberi topping strawberry dan scoop es krim vanila, dan ku 

## Word Normalization

In [ ]:
# Download corpus singkatan
!wget https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv

--2022-06-15 13:39:54--  https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80969 (79K) [text/plain]
Saving to: ‘key_norm.csv.2’

key_norm.csv.2      100%[===================>]  79.07K  --.-KB/s    in 0.01s   

2022-06-15 13:39:54 (6.39 MB/s) - ‘key_norm.csv.2’ saved [80969/80969]



In [ ]:
key_norm = pd.read_csv('key_norm.csv')

def text_normalize(mess):
  mess = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in mess.split()])
  mess = str.lower(mess)
  return mess

## Filtering (Stopword Removal)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopwords_ind = stopwords.words('indonesian')

In [ ]:
print(stopwords_ind)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [ ]:
with open('stopword.pkl', 'wb') as f:
  pickle.dump(stopwords_ind, f)

In [ ]:
# Lihat daftar stopword yang disediakan NLTK
stopwords_ind

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
# Buat fungsi untuk langkah stopword removal

def remove_stop_words(mess):
  clean_words = []
  mess = mess.split()
  for word in mess:
      if word not in stopwords_ind:
          clean_words.append(word)
  return " ".join(clean_words)

In [ ]:
raw_sample = df_review['review_text'].iloc[5]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)

Raw data		:   Ada tempat nongkrong di daerah Tubagus Ismail nih. Namanya Terai Coffeatery. Tempatnya ada di pinggir jalan banget. Menunya macem-macem. Dari yang minuman kopi hingga smoothie. Makanan beratnya juga ada lhoh. Menu yang dipesan ada blue sky, kopi susu terai, black sabbath matters, smoothies strawberry, white lychee, newyork burger, gyudon, fish and chips, triple decker sandwich. Yang paling the best adalah gyudon, nasinya pulen, beefnya berasa bumbunya.  But overall oke sih rasanya.   
Case folding		:  ada tempat nongkrong di daerah tubagus ismail nih namanya terai coffeatery tempatnya ada di pinggir jalan banget menunya macemmacem dari yang minuman kopi hingga smoothie makanan beratnya juga ada lhoh menu yang dipesan ada blue sky kopi susu terai black sabbath matters smoothies strawberry white lychee newyork burger gyudon fish and chips triple decker sandwich yang paling the best adalah gyudon nasinya pulen beefnya berasa bumbunya  but overall oke sih rasanya
Stopword rem

In [ ]:
# Buat fungsi untuk menggabungkan seluruh langkah text preprocessing
def text_preprocessing_process(mess):
  mess = casefolding(mess)
  mess = text_normalize(mess)
  mess = remove_stop_words(mess)
  return mess

In [ ]:
%%time
df_review['review_text'] = df_review['review_text'].apply(text_preprocessing_process)

# Perhatikan waktu komputasi ketika proses text preprocessing

CPU times: user 7min 29s, sys: 15.1 s, total: 7min 44s
Wall time: 7min 28s


In [ ]:
#Split train test for testing the model later
vld_size=0.15
X_train, X_valid, y_train, y_valid = train_test_split(df_review['review_text'], df_review['resto_id'], test_size = vld_size)

In [ ]:
X_train

1771    sih enaknya kawasan kampus jajanan murah minum...
5366    pilihan aman dimanapun kehausan sih sih dimana...
4522    kentang goreng nih lucu ya lihat nya nya lumay...
5059    bicara seger mengalahkan semangkuk es serut an...
5269    qboat salah makan buka daerah pasteur bandung ...
                              ...                        
1943    makan steamboat yakiniku resto all eat uang ka...
7280    salah makanan enak bandung empal gentong pingg...
2067    senang banget menemui coffeeshop gemas iniiii ...
6396    orang tau special menunya coba deh pesan menu ...
1423    siang makan nih cobain cafe lihat instagram ba...
Name: review_text, Length: 7407, dtype: object

In [ ]:
y_train

1771    16424238604236570000
5366    16424267514688690000
4522    16424211481912700000
5059    16424219246426150000
5269    16424264865931340000
                ...         
1943    16424242562609390000
7280    16424007380846920000
2067    16424244742080740000
6396    16424002230184680000
1423    16424235285888340000
Name: resto_id, Length: 7407, dtype: object

In [ ]:
X_valid

8576    malmingan komandan nonton gundala ku diajakin ...
8125    weekend enak sarapan apadaya bangunnya telat b...
7161    tampilan makanannya prasmanan macemnya gepukny...
7800    notice notifoodcation food chocolate croissant...
1069    pergi bandung nyobain namanya batagor berasa k...
                              ...                        
6870    tau cuanki kuliah bandungsetiap kesini orang b...
6961    tempatnya lumayan luas mejanyapilihan menu iga...
1284    suka nonton drama korea suka lihat adegan maka...
727     kesini ramai rangka gathering internal kantorg...
7933    kue nya enak recommended banget cheese cake ny...
Name: review_text, Length: 1308, dtype: object

In [ ]:
y_valid

8576    16424021960345850000
8125    16424016902824510000
7161    16424006461875090000
7800    16424013643657350000
1069    16424232831162660000
                ...         
6870    16424004712371520000
6961    16424005177825130000
1284    16424234150287680000
727     16424230960965750000
7933    16424014689836430000
Name: resto_id, Length: 1308, dtype: object

In [ ]:
userid_df = df_review[['user_id','review_text']]
restoid_df = df_review[['resto_id', 'review_text']]

In [ ]:
userid_df

,user_id,review_text
0,whidiea_wede,tempatnya suka pilihan sofa kursi perokok ruan...
1,laela_marlina,coba kulineran malammalam daerah tubagus ismai...
2,ExpelliarFood,ultahnya ribegitu disambut kang parkir ramah m...
3,henie_herliani,gyudonnya enak bumbunya porsinya mengenyangkan...
4,nyayu_ista,siang lunch terai coffeatery jalan tubagus ism...
...,...,...
8710,sandya_anggraswari,skeptis rice box sejenis kurangnya mamayu surp...
8711,rusliani_lia,sumpah enak banget lokasinya jalan riau bekas ...
8712,gracenathh,salted egg populer banget mamayu kitchen ngelu...
8713,nesyaadenisaa,sore nyoba salah makan hits daerah riau mamayu...


In [ ]:
userid_df[userid_df['user_id']=='nyayu_ista']['review_text']

4       siang lunch terai coffeatery jalan tubagus ism...
312     nyoba beli milkshake banget kampus tempatnya l...
538     sore sore hujan enaknya makan bakso ya pesan m...
583     nyoba lunch kampus pesan peach lychee juicenya...
592     lokasinya cimenyan dago akses jalannya dago pa...
679     kali kesini sih kelupaan tinjau oke kali kesin...
802     sebenernya sih penasaran makmur jaya namanya e...
868     nyobain ngopi kanay kupon pergikuliner tempatn...
873     coffeeshop menurutku hidden gem sih orang tau ...
904     tempatin jurnal risa dulunya mimiti coffee are...
953     lunch nyoba saikoro beef tsukemennya shin men ...
1043    sempet nyobain tempatnya banget kampus tempatn...
1098    ngopi sore kali kyomi space dago lokasinya com...
1151    lunch nyoba artisan fried chickennya jatinango...
1218    lumayan coba new yorker pizza pizza harganya k...
1231    nangkring makan enak banget sih lokasinya bagu...
1381    coffee shop siang nyoba loco lokasinya bukit d...
1815    ngopi 

In [ ]:
restoid_df.head()

,resto_id,review_text
0,16424228176488780000,tempatnya suka pilihan sofa kursi perokok ruan...
1,16424228176488780000,coba kulineran malammalam daerah tubagus ismai...
2,16424228176488780000,ultahnya ribegitu disambut kang parkir ramah m...
3,16424228176488780000,gyudonnya enak bumbunya porsinya mengenyangkan...
4,16424228176488780000,siang lunch terai coffeatery jalan tubagus ism...


In [ ]:
userid_df = userid_df.groupby('user_id').agg({'review_text': ' '.join})
restoid_df = restoid_df.groupby('resto_id').agg({'review_text': ' '.join})

In [ ]:
userid_df.head()


,review_text
user_id,
50mm.foodies,cafe asik bandung nama nya the parlor lokasi n...
Adila84,berkunjung summerbird cafe hotel tempatnya nya...
AirinAgustina,pancakerasanya banget beli dimana manacuma tte...
Alexandrasha_,tk banget kesini nyaman banget pizzanya konsis...
AlexiaOviani,ig gakenyangkenyang bentukannya sederhana bang...


In [ ]:
userid_df.loc['whidiea_wede']['review_text']


'tempatnya suka pilihan sofa kursi perokok ruangan semi outdoor depannya makanannya enak minumannya fasilitas toilet musholla nya bersih kesini parkir mobilnya luas saranin pakai kendaraan roda biar nyaman susah sih nyari ngopi initempatnya persis moxy hotel dagotempatnya asikkbersihtenangdan pilihan kopinyauntuk area smoking lantai area nomor smoking lantai wifi nya cepat lagijadi betah berlama disiniselain disedian beragam permainan uno dllnyasaya pesan ice coffee grind joerasa kopinya passs paitsusunya senang banget mampir cafe isinyaa anak muda tua ajamakanannya beragam banget bikin kantong kempes kokciyuss dehdisini banget spot foto keren berbau korea love koreaaaa visit my ig amaya_wede masuk kesini diwajibkan bayar rb voucher makanan minuman seharga segitukalau yaa bertambah deh hehesaya pilih roti bakar kejurasa rotinya enakkejunya lumayan rotinya empukuntuk teh manis hangat nya good taste angin sepoi sepoi senengnyaaa makan steak justusdagingnya empukksayurannya banyakkentangn

In [ ]:
restoid_df.loc['16424228176488780000']['review_text']


'tempatnya suka pilihan sofa kursi perokok ruangan semi outdoor depannya makanannya enak minumannya fasilitas toilet musholla nya bersih kesini parkir mobilnya luas saranin pakai kendaraan roda biar nyaman coba kulineran malammalam daerah tubagus ismail cafe terai susah parkir mobil sesampainya tempatnya lumayan cozy nyaman wfh menurutku kesana pengunjung didominasi anakanak nongkrong orangorang fokus laptop tempatnua nyaman rapi bersih pegawainya ramah banget dijelasin detail menunya menu sold pesan menumenu lae pesan white lychee chicken cordon bleu chicken quesadilla minumannya standar daaaan makanannya enakenak pesan chicken cordon bleu ekspetasi cobain kentangnya enak banget kering soft panggangbakarnya pas ukuran ayamnya pelit isiannya padat sayurannya pegawainya bilang menu salah best sellernya chicken qusadilla camilan alaala meksiko isiannya suwiran daging ayam paprikanya anak balita suka dioles sausnya enak kayanya nkesini pesan cafe aplikasi perkul pergi voucher perkul terim

##TF IDF VEctorizer##

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['review_text'])
userid_vectors.shape

(985, 5000)

In [ ]:
restoid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
restoid_vectors = restoid_vectorizer.fit_transform(restoid_df['review_text'])
restoid_vectors.shape

(1872, 5000)

##Matrix Factorization##

In [ ]:
userid_rating_matrix = pd.pivot_table(df_review, values='overall_rating', index=['user_id'], columns=['resto_id'])
userid_rating_matrix.shape


(985, 1872)

In [ ]:
userid_rating_matrix


resto_id,16423998716326680000,16423998859501140000,16423998964469800000,16423999173923800000,16423999254826950000,16423999341209980000,16423999404961350000,16423999485064210000,16423999565217630000,16423999607506610000,...,16424281971197010000,16424282028905080000,16424282070236170000,16424282105727200000,16424282126172250000,16424282147006600000,16424282199134720000,16424282242693550000,16424282297810870000,16424282343498870000
user_id,,,,,,,,,,,,,,,,,,,,,
50mm.foodies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adila84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AirinAgustina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alexandrasha_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AlexiaOviani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yvng_sagittario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zahriah_nanda_mayasari,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zakyakbar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(restoid_vectors.toarray(), index=restoid_df.index, columns=restoid_vectorizer.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
P.head()

,*,-,_,a,aa,abal,abang,abangnya,abisin,abisnya,...,yuk,yum,yumaju,yumie,yummy,yumnya,yup,yuzu,zaman,zonk
user_id,,,,,,,,,,,,,,,,,,,,,
50mm.foodies,0.0,0.038886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adila84,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AirinAgustina,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alexandrasha_,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AlexiaOviani,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Q.head()

,*,-,_,a,aa,abal,abang,abangnya,abisin,abisnya,...,yuk,yum,yumaju,yumie,yummy,yumnya,yup,yuzu,zaman,zonk
resto_id,,,,,,,,,,,,,,,,,,,,,
16423998716326680000,0.0,0.000000,0.0,0.093427,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
16423998859501140000,0.0,0.006697,0.0,0.007391,0.0,0.0,0.0,0.0,0.012820,0.0,...,0.036454,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
16423998964469800000,0.0,0.009629,0.0,0.047827,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.026629,0.0,0.0,0.0,0.000000,0.0
16423999173923800000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.013837,0.000000,0.0,0.0,0.028117,0.0,0.0,0.0,0.000000,0.0
16423999254826950000,0.0,0.046360,0.0,0.000000,0.0,0.0,0.0,0.0,0.022188,0.0,...,0.015773,0.019196,0.0,0.0,0.016025,0.0,0.0,0.0,0.020487,0.0


##Gradient Decent Optimization

In [ ]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q,e

In [ ]:
%%time
P, Q, e = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

CPU times: user 38min 4s, sys: 35.5 s, total: 38min 40s
Wall time: 37min 51s


In [ ]:
resto_ids = list(df_review['resto_id'].unique())
user_ids = list(df_review['user_id'].unique())

In [ ]:
rmse = np.sqrt(e/(len(user_ids)*len(resto_ids)))
print("RMSE Error: ",round(rmse,3))

RMSE Error:  0.136


In [ ]:
Q.head()

,*,-,_,a,aa,abal,abang,abangnya,abisin,abisnya,...,yuk,yum,yumaju,yumie,yummy,yumnya,yup,yuzu,zaman,zonk
resto_id,,,,,,,,,,,,,,,,,,,,,
16423998716326680000,0.004135,0.003864,0.000294,0.132312,0.000940,-0.000210,0.001774,0.000089,0.000175,0.000101,...,0.002101,0.000984,0.000156,0.000008,0.004333,-0.000006,0.001028,0.000360,0.000189,0.003153
16423998859501140000,0.003709,0.030863,0.000205,0.061213,0.000202,0.005388,0.002791,-0.000285,0.014273,-0.000129,...,0.079835,0.004800,0.001327,0.000049,0.007165,0.000029,-0.000231,0.001349,-0.000237,0.001922
16423998964469800000,0.005072,0.023285,0.006763,0.115713,-0.002145,-0.001721,-0.000662,-0.002439,-0.000160,-0.000868,...,0.004860,0.000702,0.000508,0.000174,0.069849,-0.000457,0.003899,-0.001537,0.000265,0.000602
16423999173923800000,0.004091,0.016004,0.000397,0.022634,0.000426,0.001498,0.006313,0.000913,0.003729,0.000441,...,0.030100,0.003846,0.003082,0.000644,0.045241,0.000288,0.002452,0.000583,0.001585,0.001746
16423999254826950000,0.021790,0.061569,0.000006,0.009107,0.002810,-0.000049,0.010214,-0.000380,0.026398,-0.000008,...,0.017699,0.039134,0.002279,0.000152,0.025208,0.000100,0.002800,0.001257,0.037019,0.002094


In [ ]:
P.head()

,*,-,_,a,aa,abal,abang,abangnya,abisin,abisnya,...,yuk,yum,yumaju,yumie,yummy,yumnya,yup,yuzu,zaman,zonk
user_id,,,,,,,,,,,,,,,,,,,,,
50mm.foodies,0.000133,0.047185,0.000055,0.010828,0.000056,0.000087,0.000252,0.000054,0.000226,0.000051,...,0.000683,0.001166,0.000256,0.000489,0.001127,0.000183,0.004373,0.000091,0.000630,0.000363
Adila84,0.000045,0.000347,0.000295,0.000242,0.000013,0.000003,0.000045,0.000046,0.000076,0.000036,...,0.000308,0.000355,0.000003,0.000004,0.000253,0.000166,0.000028,0.000007,0.000017,0.000082
AirinAgustina,0.000247,0.003236,0.000038,0.003640,0.000366,0.000110,0.000372,0.000137,0.000266,0.000246,...,0.000422,0.000385,0.000129,0.000401,0.005706,0.000015,0.000193,0.000088,0.000181,0.000300
Alexandrasha_,0.000105,0.004757,0.000037,0.001669,0.000176,0.006992,0.000210,0.000277,0.000092,0.008787,...,0.000440,0.000328,0.000234,0.000054,0.001235,0.000047,0.000180,0.000126,0.000507,0.000301
AlexiaOviani,0.000130,0.004761,0.000096,0.003569,0.000273,0.000211,0.000341,0.000317,0.001857,0.000130,...,0.000440,0.000408,0.000178,0.000042,0.004273,0.000051,0.000331,0.000210,0.000862,0.000402


In [ ]:
Q.iloc[0].sort_values(ascending=False).head(10)

nya          0.457358
banget       0.425155
enak         0.315886
and          0.278412
tempatnya    0.258648
kopi         0.246233
pesan        0.227534
is           0.224174
luas         0.221736
the          0.221316
Name: 16423998716326680000, dtype: float64

In [ ]:
Q.head()

,*,-,_,a,aa,abal,abang,abangnya,abisin,abisnya,...,yuk,yum,yumaju,yumie,yummy,yumnya,yup,yuzu,zaman,zonk
resto_id,,,,,,,,,,,,,,,,,,,,,
16423998716326680000,0.004135,0.003864,0.000294,0.132312,0.000940,-0.000210,0.001774,0.000089,0.000175,0.000101,...,0.002101,0.000984,0.000156,0.000008,0.004333,-0.000006,0.001028,0.000360,0.000189,0.003153
16423998859501140000,0.003709,0.030863,0.000205,0.061213,0.000202,0.005388,0.002791,-0.000285,0.014273,-0.000129,...,0.079835,0.004800,0.001327,0.000049,0.007165,0.000029,-0.000231,0.001349,-0.000237,0.001922
16423998964469800000,0.005072,0.023285,0.006763,0.115713,-0.002145,-0.001721,-0.000662,-0.002439,-0.000160,-0.000868,...,0.004860,0.000702,0.000508,0.000174,0.069849,-0.000457,0.003899,-0.001537,0.000265,0.000602
16423999173923800000,0.004091,0.016004,0.000397,0.022634,0.000426,0.001498,0.006313,0.000913,0.003729,0.000441,...,0.030100,0.003846,0.003082,0.000644,0.045241,0.000288,0.002452,0.000583,0.001585,0.001746
16423999254826950000,0.021790,0.061569,0.000006,0.009107,0.002810,-0.000049,0.010214,-0.000380,0.026398,-0.000008,...,0.017699,0.039134,0.002279,0.000152,0.025208,0.000100,0.002800,0.001257,0.037019,0.002094


In [ ]:
# Store P, Q and vectorizer in pickle file
output = open('bebas.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()

In [ ]:
with open('bebas.pkl', 'rb') as f:
  P=pickle.load(f)
  Q=pickle.load(f)
  userid_vectorizer=pickle.load(f)

In [ ]:
print(userid_vectorizer)

TfidfVectorizer(max_features=5000,
                tokenizer=<bound method RegexpTokenizer.tokenize of WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)


In [ ]:
words = "makanan murah di bandung"
test_df= pd.DataFrame([words], columns=['review_text'])
test_df['review_text'] = test_df['review_text'].apply(text_preprocessing_process)
test_vectors = userid_vectorizer.transform(test_df['review_text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:10]

for i in topRecommendations.index:
    print(df_tempat[df_tempat['resto_id']==i]['name'].iloc[0])
    print(df_tempat[df_tempat['resto_id']==i]['overall_rating'].iloc[0])
    print('')

Nice Day Coffee
4.4

One Eighty Coffee and Music
4.11

Warung Nasi Mandiri
3.0

Kokio
2.8

Penyetan Ibu Suwarti
3.8

Ambrogio Patisserie
4.19

Eat Boss
3.95

Kopi Endeus
4.0

RM Diwo
3.6

Chingu Korean Fan Cafe
3.97



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
topRecommendations.index

Index(['16424235585998100000', '16424228305359590000', '16424247365316870000',
       '16424275833230580000', '16424243608292770000', '16423998964469800000',
       '16424229380353520000', '16424281099909660000', '16424237506453980000',
       '16424228881536780000'],
      dtype='object', name='resto_id')

In [ ]:
def recommend(words):
  test_df= pd.DataFrame([words], columns=['review_text'])
  test_df['review_text'] = test_df['review_text'].apply(text_preprocessing_process)
  test_vectors = userid_vectorizer.transform(test_df['review_text'])
  test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names_out())
  
  predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
  topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:10]
  recommended_place = []
  for i in topRecommendations.index:
    recommended_place.append(df_tempat[df_tempat['resto_id']==i]['name'].iloc[0])
  return recommended_place

In [ ]:
recommend('makanan')

["Cafe D'Pakar",
 'Cafe Tulip',
 'One Eighty Coffee and Music',
 'Ambrogio Patisserie',
 'Jenderal Kopi Nusantara Buwas',
 'Kapulaga Indonesian Bistro',
 'The Kamasan',
 'Gudeg Bu Ratna',
 'Sangucitel',
 'Toko You']

In [ ]:
pickle.dump(df_tempat,open('df_tempat.pkl','wb'))

In [ ]:
pickle.dump(df_review,open('df_review.pkl','wb'))